### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

Installing Required Packages

In [1]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
# import cudf
# import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Import

In [5]:
import oci
import pandas as pd
import io
import concurrent.futures

# Set up OCI config, the values are generated upon generating API key
config = {
    "user": "ocid1.user.oc1..aaaaaaaabepmp6cnh7quc2kq2lp65bg5zoaaafb3i6ysliw4xtsolf5245pq", 
    "key_file": "secret_key/swapnil.roy@infolob.com_2025-02-28T11_09_10.611Z.pem",
    "tenancy": "ocid1.tenancy.oc1..aaaaaaaamvz6uy5l5mdihzdqex43sp3kwdg45hzhm5djvfiqf2aq2fusbb4q", 
    "fingerprint": "54:d0:cd:22:ea:f1:2f:ca:01:62:de:58:57:0a:7d:76",
    "region": "ap-mumbai-1"
}

# Create Object Storage client
object_storage = oci.object_storage.ObjectStorageClient(config)

# Bucket and namespace details
namespace = "bmglwagxa8my"  
bucket_name = "ppepoc_landing"
folder_prefix = "ascp_cleaned_data/"  # Folder inside the bucket

# Get list of CSV files in the bucket
file_list = [
    obj.name for obj in object_storage.list_objects(namespace, bucket_name, prefix=folder_prefix).data.objects
    if obj.name.endswith(".csv")
]

print(f"Found {len(file_list)} CSV files in {folder_prefix}")

# Function to fetch and read CSV as a DataFrame
def get_csv_as_dataframe(object_name):
    try:
        obj = object_storage.get_object(namespace, bucket_name, object_name)
        return pd.read_csv(io.BytesIO(obj.data.content))
    except Exception as e:
        print(f"Error loading {object_name}: {e}")
        return None

# Read CSVs in parallel using ThreadPoolExecutor
df_list = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(get_csv_as_dataframe, file_list))

# Remove None values (failed loads)
df_list = [df for df in results if df is not None]

# Combine all DataFrames
if df_list:
    data = pd.concat(df_list, ignore_index=True)
    print("Combined DataFrame Shape:", data.shape)

    # Basic insights
    print("Unique ORG values:", data["ORG"].nunique())
    print("Unique ITEM_CODE values:", data["ITEM_CODE"].nunique())
else:
    print("No valid CSV files found!")


Found 37 CSV files in ascp_cleaned_data/
Combined DataFrame Shape: (27729179, 9)
Unique ORG values: 36
Unique ITEM_CODE values: 69044


In [6]:
# Dropping duplicate records
# data = data.drop_duplicates()
data.drop_duplicates(inplace=True)

In [7]:
print(data.shape)

(27595574, 9)


### SAMPLE: Data Sample Export (item_code == '02150513')

# Data Manipulation

In [8]:
# Splitting CATEGORY_NAME into four new columns
split_cols = data['CATEGORY_NAME'].str.split('.', expand=True)
split_cols.columns = ['CN_1', 'CN_2', 'CN_3', 'CN_4']

# Efficiently concatenate the new columns after CATEGORY_NAME
col_position = data.columns.get_loc('CATEGORY_NAME') + 1  # Position after CATEGORY_NAME
data = pd.concat([data.iloc[:, :col_position], split_cols, data.iloc[:, col_position:]], axis=1)

## Converting all column names to lower case

In [9]:
data.columns = data.columns.str.lower()

## Converting columns to appropriate data type

In [10]:
data['collection_date'] = pd.to_datetime(data['collection_date'])
data = data.astype({'recommended_total_qty':'float'})

In [11]:
data['cn_4'] = data['cn_4'].fillna('NA')

## Maintaining an instance of full raw data

In [12]:
data_full = data.copy(deep=True)

In [13]:
data_full['item_code'].nunique()

69044

In [14]:
data_full[data_full['recommended_total_qty']>0].shape[0]/data_full.shape[0]

0.10296114876972662

In [15]:
data_full['cn_3'].unique()

array(['IMP', 'INDG', 'CSI'], dtype=object)

# Data Filtration

## Removing all zero recommended_total_qty for the ease of analyses

In [16]:
data = data[data['recommended_total_qty'] > 0]

### SAMPLE: min & max collection_date for each Plant

## CONDITION 1(ACTIVE ITEMS): Items ordered (non-zero) atleast once in past 3 months (at plant level considering global "max_date")

### Calculating global max_date

In [17]:
max_date = data['collection_date'].max()
active_item_lookback = max_date - pd.DateOffset(months = 3)

In [18]:
print(max_date, active_item_lookback)

2025-03-03 23:05:23 2024-12-03 23:05:23


In [19]:
def active_item_filter(df, lookback):
    df_list = []
    df = df.copy()
    lookback_dt = max_date - pd.DateOffset(months = lookback)
    for org in data['org'].unique():
        df_org = df[df['org']==org]
        filtered_items = df_org[(df_org['collection_date']>=lookback_dt) & (df_org['collection_date']<=max_date)]['item_code'].unique()
        df_org = df_org[df_org['item_code'].isin(filtered_items)]
        if df_org.shape[0]>0:
            df_list.append(df_org)
            print(f'org: {df_org["org"].unique()} :: Items: {df_org["item_code"].nunique()} :: Max order date: {df_org["collection_date"].max()}')
        else:
            continue
    print(f'Total Plants: {len(df_list)}')
    return df_list

In [20]:
data_list = active_item_filter(data, 3) #(dataframe, lookback in months)

org: ['P01'] :: Items: 474 :: Max order date: 2025-02-02 14:02:57
org: ['P02'] :: Items: 3145 :: Max order date: 2025-02-22 00:50:19
org: ['P03'] :: Items: 2369 :: Max order date: 2025-02-22 00:50:19
org: ['P04'] :: Items: 665 :: Max order date: 2025-02-02 14:02:57
org: ['P07'] :: Items: 428 :: Max order date: 2025-02-02 14:02:57
org: ['P08'] :: Items: 545 :: Max order date: 2025-02-02 14:02:57
org: ['P09'] :: Items: 178 :: Max order date: 2025-02-02 14:02:57
org: ['P10'] :: Items: 519 :: Max order date: 2025-03-03 23:05:23
org: ['P11'] :: Items: 670 :: Max order date: 2025-02-08 03:11:39
org: ['P12'] :: Items: 526 :: Max order date: 2025-02-02 14:02:57
org: ['P14'] :: Items: 764 :: Max order date: 2025-02-08 03:11:39
org: ['P15'] :: Items: 915 :: Max order date: 2025-02-08 03:11:39
org: ['P16'] :: Items: 1474 :: Max order date: 2025-02-08 03:11:39
org: ['P18'] :: Items: 5 :: Max order date: 2024-12-22 22:20:10
org: ['P20'] :: Items: 313 :: Max order date: 2025-02-08 03:11:39
org: ['P2

### Concatanating the dataframes back into a single one

In [21]:
f_data = pd.concat(data_list, axis=0)

## CONDITION 2(ORDER FREQUENCY THRESHOLD): Items ordered (non-zero) atleast X times in past Y months (at plant level considering global "max_date")

In [22]:
def order_freq_filter(df, threshold, lookback):
    df_list = []
    df = df.copy()
    lookback_dt = max_date - pd.DateOffset(months = lookback)
    # for org in ['P01']:
    for org in data['org'].unique():
        df_org = df[df['org']==org]
        
        df_org_temp = df_org[(df_org['collection_date']>=lookback_dt) & (df_org['collection_date']<=max_date)]
        df_org_temp_item_order_freq = df_org_temp.groupby('item_code').size().reset_index(name="order_frequency")
        filtered_items = df_org_temp_item_order_freq[df_org_temp_item_order_freq["order_frequency"] >= threshold]["item_code"].unique()
        
        df_org = df_org[df_org["item_code"].isin(filtered_items)]
        
        if df_org.shape[0]>0:
            df_list.append(df_org)
            print(f'org: {df_org["org"].unique()} :: Items: {df_org["item_code"].nunique()}')
        else:
            continue
    
    print(f'Total Plants: {len(df_list)}')
    return df_list

In [23]:
data_list = order_freq_filter(f_data, 10, 12)

org: ['P01'] :: Items: 77
org: ['P02'] :: Items: 1274
org: ['P03'] :: Items: 759
org: ['P04'] :: Items: 108
org: ['P07'] :: Items: 163
org: ['P08'] :: Items: 104
org: ['P09'] :: Items: 17
org: ['P10'] :: Items: 221
org: ['P11'] :: Items: 78
org: ['P12'] :: Items: 98
org: ['P14'] :: Items: 80
org: ['P15'] :: Items: 524
org: ['P16'] :: Items: 12
org: ['P22'] :: Items: 22
org: ['P23'] :: Items: 1
org: ['P31'] :: Items: 512
org: ['P32'] :: Items: 869
org: ['P34'] :: Items: 751
org: ['P36'] :: Items: 39
org: ['P40'] :: Items: 586
org: ['P41'] :: Items: 83
org: ['P42'] :: Items: 103
Total Plants: 22


### Collating data_list to single dataframe

In [24]:
f_data2 = pd.concat(data_list, axis=0)

In [25]:
f_data2['org'].nunique()

22

In [26]:
f_data2.shape

(290072, 13)

In [27]:
f_data2.head()

,org,item_code,category_name,cn_1,cn_2,cn_3,cn_4,description,uom,run_period,collection_date,run_no,recommended_total_qty
635,P01,14000536,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,218734-2L 187,Nos,21-FEB-21 - 07-NOV-21,2021-02-25 09:19:16,245,45.0
900,P01,19004458,RAWM.HOUS.IMP.NA,RAWM,HOUS,IMP,NA,50 WAY F W/P HOUS ASSY BLK,Nos,12-APR-20 - 27-DEC-20,2020-04-11 04:28:48,186,3511.0
999,P01,18002350,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,TRML F 250 POSI LK(CSA 0.75-2.0),Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,9000.0
1029,P01,64400297,RAWM.CONN.IMP.NA,RAWM,CONN,IMP,NA,HX4J-3/12-G,Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,51.0
1043,P01,60984979,RAWM.HOUS.IMP.NA,RAWM,HOUS,IMP,NA,HE02 6F-NA,Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,1412.0


## Data Export (Exporting filtered dataframe)

# Data Cleaning

In [28]:
def fix_anomalous_values(f_data, columns_to_fix):
    """
    Identifies and fixes anomalies in specified columns by assigning 
    the latest available value based on the most recent collection_date.

    Parameters:
    f_data (pd.DataFrame): The procurement dataset.
    columns_to_fix (list): List of column names to check and fix (e.g., ['cn_4', 'cn_2', 'description']).

    Returns:
    pd.DataFrame: The cleaned dataset with anomalies fixed.
    """
    f_data = f_data.copy()  # Avoid modifying the original dataframe

    # Step 1: Identify all affected item_codes for given columns
    affected_item_codes = set()
    
    for col in columns_to_fix:
        t = f_data.drop_duplicates(subset=['item_code', col])
        t2 = t.groupby('item_code')[col].nunique().reset_index(name=f'unique_{col}_count')
        anomalous_items = t2[t2[f'unique_{col}_count'] > 1]['item_code'].unique()
        affected_item_codes.update(anomalous_items)

    # Step 2: Get the latest values for affected item_codes
    latest_values = f_data[f_data['item_code'].isin(affected_item_codes)] \
                    .sort_values(by=['item_code', 'collection_date'], ascending=[True, False]) \
                    .drop_duplicates(subset=['item_code'])[['item_code'] + columns_to_fix]

    # Step 3: Convert latest values into mapping dictionaries
    latest_value_dicts = {col: dict(zip(latest_values['item_code'], latest_values[col])) for col in columns_to_fix}

    # Step 4: Apply fixes in the original data
    for col in columns_to_fix:
        f_data.loc[f_data['item_code'].isin(affected_item_codes), col] = f_data['item_code'].map(latest_value_dicts[col])

    return f_data

In [29]:
columns_to_fix = ['cn_4', 'cn_2', 'description']
f_data_cleaned = fix_anomalous_values(f_data2, columns_to_fix)

In [30]:
print(f_data2.shape, f_data_cleaned.shape)

(290072, 13) (290072, 13)


In [31]:
f_data_cleaned.head()

,org,item_code,category_name,cn_1,cn_2,cn_3,cn_4,description,uom,run_period,collection_date,run_no,recommended_total_qty
635,P01,14000536,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,218734-2L 187,Nos,21-FEB-21 - 07-NOV-21,2021-02-25 09:19:16,245,45.0
900,P01,19004458,RAWM.HOUS.IMP.NA,RAWM,HOUS,IMP,NA,50 WAY F W/P HOUS ASSY BLK,Nos,12-APR-20 - 27-DEC-20,2020-04-11 04:28:48,186,3511.0
999,P01,18002350,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,TRML F 250 POSI LK(CSA 0.75-2.0),Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,9000.0
1029,P01,64400297,RAWM.CONN.IMP.NA,RAWM,CONN,IMP,NA,HX4J-3/12-G,Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,51.0
1043,P01,60984979,RAWM.HOUS.IMP.NA,RAWM,HOUS,IMP,NA,HE02 6F-NA,Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,1412.0


## Data Export (Exporting filtered dataframe cleaned and features consistent at item level)

# Checkpoint 1 (Filtered Cleaned Data)

## Dropping columns irrelevant for forecasting

In [32]:
f_data_cleaned.head(3)

,org,item_code,category_name,cn_1,cn_2,cn_3,cn_4,description,uom,run_period,collection_date,run_no,recommended_total_qty
635,P01,14000536,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,218734-2L 187,Nos,21-FEB-21 - 07-NOV-21,2021-02-25 09:19:16,245,45.0
900,P01,19004458,RAWM.HOUS.IMP.NA,RAWM,HOUS,IMP,NA,50 WAY F W/P HOUS ASSY BLK,Nos,12-APR-20 - 27-DEC-20,2020-04-11 04:28:48,186,3511.0
999,P01,18002350,RAWM.TRML.IMP.NA,RAWM,TRML,IMP,NA,TRML F 250 POSI LK(CSA 0.75-2.0),Nos,07-MAR-20 - 22-NOV-20,2020-03-06 02:30:29,182,9000.0


In [33]:
model_data = f_data_cleaned.drop(columns=["category_name","cn_1","cn_3","cn_4","description","run_period","run_no"])

In [34]:
model_data['uom'] = model_data['uom'].str.upper()
model_data['cn_2'] = model_data['cn_2'].str.upper()
model_data['uom'] = model_data['uom'].str.strip()

In [35]:
model_data.head()

,org,item_code,cn_2,uom,collection_date,recommended_total_qty
635,P01,14000536,TRML,NOS,2021-02-25 09:19:16,45.0
900,P01,19004458,HOUS,NOS,2020-04-11 04:28:48,3511.0
999,P01,18002350,TRML,NOS,2020-03-06 02:30:29,9000.0
1029,P01,64400297,CONN,NOS,2020-03-06 02:30:29,51.0
1043,P01,60984979,HOUS,NOS,2020-03-06 02:30:29,1412.0


In [36]:
model_data['uom'].value_counts()

uom
NOS    205784
M       80701
KG       2074
L        1513
Name: count, dtype: int64

In [37]:
model_data['item_code'].nunique()

5396

## Plant level complete week filled dataframe for each items

In [38]:
def assign_week_number(f_data):
    #f_data = f_data.copy()
    """
    Assigns a week number to each record based on the minimum collection_date in the dataset.

    Parameters:
    f_data (pd.DataFrame): The dataset with a 'collection_date' column.

    Returns:
    pd.DataFrame: The dataset with an added 'week_number' column.
    """
    # Convert collection_date to datetime if it's not already
    # f_data['collection_date'] = pd.to_datetime(f_data['collection_date'])

    # Find the minimum collection date
    min_date = f_data['collection_date'].min()

    # Calculate the difference in days and assign week numbers
    f_data['week_number'] = ((f_data['collection_date'] - min_date).dt.days // 7) + 1

    return f_data


def roll_up_weekly(f_data):
    """
    Aggregates the dataset at the item-week level by summing recommended_total_qty.

    Parameters:
    f_data (pd.DataFrame): The dataset with 'item_code', 'week_number', and 'recommended_total_qty'.

    Returns:
    pd.DataFrame: A rolled-up dataset at the item-week level.
    """
    # Ensure week_number is computed
    if 'week_number' not in f_data.columns:
        raise ValueError("week_number column is missing. Ensure it is computed before rolling up.")

    # Define columns to keep (ensuring we retain unique values per item)
    columns_to_keep = ['org','item_code', 'cn_2', 'uom']

    # Drop duplicates to retain only one unique row per item
    f_data_unique = f_data.drop_duplicates(subset=['item_code'])[columns_to_keep]

    # Aggregate at item-week level
    f_data_agg = f_data.groupby(['item_code', 'week_number'], as_index=False)['recommended_total_qty'].sum()

    # Merge aggregated values back with the unique item-level details
    f_data_final = pd.merge(f_data_agg, f_data_unique, on='item_code', how='left')

    return f_data_final


def prepare_weekly_forecasting_data(f_data):
    """
    Prepares the weekly forecasting dataset by rolling up data at the week level, 
    filling missing weeks with zero demand, and ensuring item metadata is retained.

    Parameters:
        f_data (pd.DataFrame): The cleaned dataset with 'item_code', 'week_number', and 'recommended_total_qty'.

    Returns:
        pd.DataFrame: Processed dataset with missing weeks filled.
    """
    # Step 1: Find the global min and max week numbers
    min_week = f_data['week_number'].min()
    max_week = f_data['week_number'].max()

    # Step 2: Get all unique item codes
    all_items = f_data['item_code'].unique()

    # Step 3: Create a full DataFrame with all combinations of (item_code, week_number)
    all_weeks = range(min_week, max_week + 1)
    full_index = pd.MultiIndex.from_product([all_items, all_weeks], names=['item_code', 'week_number'])
    full_df = pd.DataFrame(index=full_index).reset_index()

    # Step 4: Merge with only the required columns to avoid duplication
    f_data_subset = f_data[['item_code', 'week_number', 'recommended_total_qty']]
    f_data_filled = pd.merge(full_df, f_data_subset, on=['item_code', 'week_number'], how='left')

    # Step 5: Fill missing values with zero for demand
    f_data_filled['recommended_total_qty'] = f_data_filled['recommended_total_qty'].fillna(0)

    # Step 6: Merge metadata from the original data, ensuring unique item mappings
    metadata_cols = ['org', 'uom', 'cn_2']
    f_data_filled = pd.merge(
        f_data_filled, 
        f_data.drop_duplicates(subset=['item_code'])[['item_code'] + metadata_cols], 
        on='item_code', 
        how='left'
    )

    return f_data_filled

In [39]:
def plant_level_operations(df):
    df = df.copy()
    df_list = []
    
    for org in df['org'].unique():
        df_org = df[df['org']==org]
        with_week_number_df = assign_week_number(df_org)
        #print(with_week_number_df.head(2))
        rolled_up_weekly_df = roll_up_weekly(with_week_number_df)
        #print(rolled_up_weekly_df.head(2))
        prepare_weekly_forecasting_data_df = prepare_weekly_forecasting_data(rolled_up_weekly_df)
        #print(prepare_weekly_forecasting_data_df.head(2))
        df_list.append(prepare_weekly_forecasting_data_df)

    return df_list

In [40]:
model_data_list = plant_level_operations(model_data)

## Info

In [41]:
counter = 0
for df in model_data_list:
    print(f'Plant: {df["org"].unique()} :: min_week: {df["week_number"].min()} :: max_week: {df["week_number"].max()} :: Items: {df["item_code"].nunique()} :: INDEX: {counter}')
    counter+=1

Plant: ['P01'] :: min_week: 1 :: max_week: 267 :: Items: 77 :: INDEX: 0
Plant: ['P02'] :: min_week: 1 :: max_week: 103 :: Items: 1274 :: INDEX: 1
Plant: ['P03'] :: min_week: 1 :: max_week: 103 :: Items: 759 :: INDEX: 2
Plant: ['P04'] :: min_week: 1 :: max_week: 267 :: Items: 108 :: INDEX: 3
Plant: ['P07'] :: min_week: 1 :: max_week: 267 :: Items: 163 :: INDEX: 4
Plant: ['P08'] :: min_week: 1 :: max_week: 267 :: Items: 104 :: INDEX: 5
Plant: ['P09'] :: min_week: 1 :: max_week: 267 :: Items: 17 :: INDEX: 6
Plant: ['P10'] :: min_week: 1 :: max_week: 105 :: Items: 221 :: INDEX: 7
Plant: ['P11'] :: min_week: 1 :: max_week: 267 :: Items: 78 :: INDEX: 8
Plant: ['P12'] :: min_week: 1 :: max_week: 267 :: Items: 98 :: INDEX: 9
Plant: ['P14'] :: min_week: 1 :: max_week: 267 :: Items: 80 :: INDEX: 10
Plant: ['P15'] :: min_week: 1 :: max_week: 267 :: Items: 524 :: INDEX: 11
Plant: ['P16'] :: min_week: 1 :: max_week: 267 :: Items: 12 :: INDEX: 12
Plant: ['P22'] :: min_week: 1 :: max_week: 267 :: Ite

In [42]:
pip install PrettyTable

Note: you may need to restart the kernel to use updated packages.


In [43]:
from prettytable import PrettyTable

# Initialize PrettyTable
table = PrettyTable()
table.field_names = ["Plant", "Min Week", "Max Week", "Items"]

# Populate table
counter = 0
for df in model_data_list:
    plant = df["org"].unique()[0]  # Assuming one unique plant per df
    min_week = df["week_number"].min()
    max_week = df["week_number"].max()
    items = df["item_code"].nunique()
    
    table.add_row([plant, min_week, max_week, items])

# Print the table
print(table)


+-------+----------+----------+-------+
| Plant | Min Week | Max Week | Items |
+-------+----------+----------+-------+
|  P01  |    1     |   267    |   77  |
|  P02  |    1     |   103    |  1274 |
|  P03  |    1     |   103    |  759  |
|  P04  |    1     |   267    |  108  |
|  P07  |    1     |   267    |  163  |
|  P08  |    1     |   267    |  104  |
|  P09  |    1     |   267    |   17  |
|  P10  |    1     |   105    |  221  |
|  P11  |    1     |   267    |   78  |
|  P12  |    1     |   267    |   98  |
|  P14  |    1     |   267    |   80  |
|  P15  |    1     |   267    |  524  |
|  P16  |    1     |   267    |   12  |
|  P22  |    1     |   267    |   22  |
|  P23  |    1     |    56    |   1   |
|  P31  |    1     |   105    |  512  |
|  P32  |    1     |   155    |  869  |
|  P34  |    1     |   136    |  751  |
|  P36  |    1     |   105    |   39  |
|  P40  |    1     |    16    |  586  |
|  P41  |    1     |    16    |   83  |
|  P42  |    1     |    16    |  103  |


In [44]:
mod_data_comb = pd.concat(model_data_list, axis=0)
print(f'Unique Items: {mod_data_comb["item_code"].nunique()} :: Plants: {mod_data_comb["org"].nunique()} :: Included_Plants: {mod_data_comb["org"].unique()}')
del mod_data_comb

Unique Items: 5396 :: Plants: 22 :: Included_Plants: ['P01' 'P02' 'P03' 'P04' 'P07' 'P08' 'P09' 'P10' 'P11' 'P12' 'P14' 'P15'
 'P16' 'P22' 'P23' 'P31' 'P32' 'P34' 'P36' 'P40' 'P41' 'P42']


# Feature Engineering

In [45]:
import numpy as np
import pandas as pd

def add_features(df, lags=[1, 2, 4, 8], rolling_windows=[2, 4, 8]):
    """
    Adds lag features, rolling statistics, time-based cyclic features, 
    and an 'is_live' feature to indicate when an item becomes active.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with columns ["item_code", "week_number", "recommended_total_qty"].
    lags (list): List of lag periods to include.
    rolling_windows (list): List of window sizes for moving averages.

    Returns:
    pd.DataFrame: DataFrame with new features added.
    """
    df = df.copy()
    
    # Sort values to ensure correct lagging
    df.sort_values(by=["item_code", "week_number"], inplace=True)
    
    # Group by item_code to ensure operations happen per item
    grouped = df.groupby("item_code")
    
    # Add lag features
    for lag in lags:
        df[f"lag_{lag}"] = grouped["recommended_total_qty"].shift(lag).fillna(0)  

    # Add rolling statistics
    for window in rolling_windows:
        df[f"rolling_avg_{window}"] = grouped["recommended_total_qty"].transform(lambda x: x.rolling(window).mean()).fillna(method='ffill').fillna(0)
        df[f"rolling_pct_change_{window}"] = grouped["recommended_total_qty"].transform(lambda x: x.pct_change(periods=window)).replace([float("inf"), float("-inf")], 1.0).fillna(0)

    # ---- TIME-BASED FEATURES ---- #
    df["week_sin"] = np.sin(2 * np.pi * df["week_number"] / 52)
    df["week_cos"] = np.cos(2 * np.pi * df["week_number"] / 52)
    
    # ---- NEW FEATURE: IS_LIVE ---- #
    def compute_is_live(series):
        first_nonzero_index = (series != 0).idxmax()  # Find first non-zero week index
        return (series.index >= first_nonzero_index).astype(int)  # 1 after first demand, 0 before

    df["is_live"] = grouped["recommended_total_qty"].transform(compute_is_live)

    return df

In [46]:
def feature_engineering(df_list):
    df_list_final = []
    
    for df in model_data_list:
        df_list_final.append(add_features(df))
        
    return df_list_final

In [47]:
model_data_list_engineered = feature_engineering(model_data_list)

In [48]:
model_data_list_engineered[0].head(1)

,item_code,week_number,recommended_total_qty,org,uom,cn_2,lag_1,lag_2,lag_4,lag_8,rolling_avg_2,rolling_pct_change_2,rolling_avg_4,rolling_pct_change_4,rolling_avg_8,rolling_pct_change_8,week_sin,week_cos,is_live
0,00950404,1,0.0,P01,M,WIRE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.120537,0.992709,0


### SAMPLE: Send one plant-item pair data post feature engineering

# XGBoost

In [49]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [50]:
def plot_feature_importance(model, feature_names):
    """
    Plots the feature importance from the trained XGBoost model.

    Parameters:
    - model: Trained XGBoost model.
    - feature_names: List of feature names used in the model.
    """
    # Get feature importance scores
    importance_values = model.feature_importances_
    
    # Create a DataFrame and sort by importance
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance_values})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # Plot
    plt.figure(figsize=(12, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'], color='royalblue')
    plt.xlabel("Feature Importance Score")
    plt.ylabel("Features")
    plt.title("XGBoost Feature Importance (Sorted)")
    plt.gca().invert_yaxis()  # Highest importance on top
    plt.show()


In [51]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

def train_xgboost(X_train, X_test, y_train, y_test, df_train, df_test, original_df):

    # Define the model # 42
    model = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=6,
        random_state=43,
        enable_categorical=True
    )

    # Train the model
    model.fit(X_train, y_train)
    
    # Generate feature importance plot
    # plot_feature_importance(model, feature_names=X_train.columns.to_list())

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Ensure no negative predictions (set negative values to 0)
    y_train_pred = np.where(y_train_pred < 0, 0, y_train_pred)
    y_test_pred = np.where(y_test_pred < 0, 0, y_test_pred)

    # Updated MAPE function (row-wise calculation)
    def mape(actual, predicted, threshold=100):
        if actual == 0 and predicted == 0:
            return 0  # Case 1: Both are zero, no error
        elif actual == 0:
            if predicted < threshold:
                return 0  # Ignore small fluctuations when AD is 0
            else:
                actual_adj = 1  # Avoid division by zero by setting AD to 1
        else:
            actual_adj = actual  # Normal case

        return 100 * abs(predicted - actual) / actual_adj

    # Prepare Results DataFrame
    df_test["predicted_demand"] = y_test_pred
    df_test["actual_demand"] = y_test
    df_test["error"] = df_test["actual_demand"] - df_test["predicted_demand"]

    # Apply the updated MAPE function row-wise
    df_test["percentage_error"] = df_test.apply(lambda row: mape(row["actual_demand"], row["predicted_demand"]), axis=1)

    # Restore original item_code (since it was encoded)
    df_test["item_code"] = original_df.loc[df_test.index, "item_code"]
    df_test["uom"] = original_df.loc[df_test.index, "uom"]
    df_test["org"] = original_df.loc[df_test.index, "org"]
    
    results_df = df_test[["org", "item_code", "actual_demand", "predicted_demand", "error", "percentage_error", "uom"]]

    # Compute overall MAPE for reporting
    metrics_dict = {
        "Train MAE": mean_absolute_error(y_train, y_train_pred),
        "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "Train MAPE": np.nanmean([mape(a, p) for a, p in zip(y_train, y_train_pred)]),  # Aggregate at dataset level
        "Test MAE": mean_absolute_error(y_test, y_test_pred),
        "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "Test MAPE": np.nanmean([mape(a, p) for a, p in zip(y_test, y_test_pred)]),  # Aggregate at dataset level
    }
    
    print(metrics_dict)
    return metrics_dict, results_df


In [52]:
from sklearn.preprocessing import StandardScaler

def preprocess_data(df, test_weeks, train_window=None):

    df = df.copy()
    original_df = df.copy()
    
    # Define features and target
    target = "recommended_total_qty"
    features = df.columns.difference(["recommended_total_qty","org"]).tolist()


    def encode_categorical(df):
        """Encodes categorical features like 'uom', 'item_code', and 'cn_2'."""
        df['uom'] = df['uom'].astype('category')
        df['item_code'] = df['item_code'].astype('category')
        df['cn_2'] = df['cn_2'].astype('category')
        df['is_live'] = df['is_live'].astype('category')
        return df

    # Encode categorical variables
    df = encode_categorical(df)

    # Get the latest week number
    max_week = df["week_number"].max()
    test_start_week = max_week - test_weeks + 1

    df_test = df[df["week_number"] >= test_start_week]  # Test set remains the same

    # Apply train window restriction
    if train_window:
        train_start_week = max(test_start_week - train_window, df["week_number"].min())  
        df_train = df[(df["week_number"] >= train_start_week) & (df["week_number"] < test_start_week)]
    else:
        df_train = df[df["week_number"] < test_start_week]  # Use all weeks before the test period

     # Debugging print statements
    # print(f"Max week: {max_week}, Test start week: {test_start_week}")
    # print(f"X_train shape: {df_train.shape}, X_test shape: {df_test.shape}")
    # print(f"Train start week: {df_train['week_number'].min()}, Train end week: {df_train['week_number'].max()}")


    # Normalize numerical variables **AFTER** train-test split
    numerical_cols = [
        "week_number", "lag_1", "lag_2", "lag_4", "lag_8", 
        "rolling_avg_2", "rolling_pct_change_2","rolling_avg_4", "rolling_pct_change_4", "rolling_avg_8", 
        "rolling_pct_change_8", "week_sin", "week_cos"
    ]
    
    
    scaler = StandardScaler()
    df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols])
    df_test[numerical_cols] = scaler.transform(df_test[numerical_cols])  # Use same scaler for test set

    # Split into X and y
    X_train, y_train = df_train[features], df_train[target]
    X_test, y_test = df_test[features], df_test[target]

    # Function call to train XGBoost with the prepared train-test split
    # train_xgboost(X_train, X_test, y_train, y_test, df_train, df_test, original_df)
    return X_train, X_test, y_train, y_test, df_train, df_test, original_df

In [53]:
def main_call(df_list, test_weeks, train_window):
    df_list = df_list.copy()
    metrics_dict_list = []
    results_df_list = []
    
    for df in df_list:
        X_train, X_test, y_train, y_test, df_train, df_test, original_df = preprocess_data(df, test_weeks=test_weeks, train_window=train_window)
        metrics_dict, results_df = train_xgboost(X_train, X_test, y_train, y_test, df_train, df_test, original_df)
        metrics_dict_list.append(metrics_dict)
        results_df_list.append(results_df)
        
    return metrics_dict_list, results_df_list

In [54]:
metrics_dict_list, results_df_list = main_call(model_data_list_engineered, test_weeks=4, train_window=None)

Max week: 267, Test start week: 264
X_train shape: (20251, 19), X_test shape: (308, 19)
Train start week: 1, Train end week: 263
{'Train MAE': 14.591435409919907, 'Train RMSE': 31.767308409714712, 'Train MAPE': 13.247239664587394, 'Test MAE': 153.78313425899327, 'Test RMSE': 2179.3945610867763, 'Test MAPE': 721.0762050053411}
Max week: 103, Test start week: 100
X_train shape: (126126, 19), X_test shape: (5096, 19)
Train start week: 1, Train end week: 99
{'Train MAE': 25.802732648959793, 'Train RMSE': 74.85681124434329, 'Train MAPE': 214.1024719225966, 'Test MAE': 621.527347358508, 'Test RMSE': 11237.42229203579, 'Test MAPE': 9686.497821649898}
Max week: 103, Test start week: 100
X_train shape: (75141, 19), X_test shape: (3036, 19)
Train start week: 1, Train end week: 99
{'Train MAE': 11.032284399541988, 'Train RMSE': 27.665062794304912, 'Train MAPE': 33.034314376919916, 'Test MAE': 312.0046525772143, 'Test RMSE': 2309.736377345554, 'Test MAPE': 7069.713340670099}
Max week: 267, Test st

# Results

In [55]:
results_df_list[0].head(1)

,org,item_code,actual_demand,predicted_demand,error,percentage_error,uom
263,P01,00950404,0.0,0.0,0.0,0.0,M


In [56]:
results_df_list[0]['item_code'].nunique()

77

In [57]:
result_df_combined = pd.concat(results_df_list, axis=0)

In [58]:
result_df_combined.to_csv(r'4_week_lead_time/XGBoost_Results_4.csv', index=False)
print("Results Exported")

Results Exported


In [59]:
import pandas as pd

def summarize_items_by_error(df):
    """
    Groups items based on their average percentage error (in 5% increments) and aggregates their counts.

    Args:
    df (pd.DataFrame): DataFrame containing 'org', 'item_code', and 'percentage_error'.

    Returns:
    pd.DataFrame: Summarized DataFrame with item counts per error bucket.
    """
    df = df.copy()

    # Step 1: Compute mean percentage error at item level
    avg_error_per_item = df.groupby(["org", "item_code"])["percentage_error"].mean().reset_index()

    # Step 2: Define percentage error bins (5% increments up to 50%, then 50%+)
    bins = list(range(0, 55, 5)) + [float('inf')]  # [0,5,10,15,...,45,50,inf]
    labels = [f"{i}-{i+5}%" for i in range(0, 50, 5)] + ["50% and above"]

    # Step 3: Assign error bucket based on mean error per item
    avg_error_per_item["pe_bucket"] = pd.cut(
        avg_error_per_item["percentage_error"], bins=bins, labels=labels, right=False
    )

    # Step 4: Count unique items per error bucket
    summary_df = avg_error_per_item.groupby(["org", "pe_bucket"])["item_code"].nunique().reset_index()
    summary_df.rename(columns={"item_code": "no_unique_item"}, inplace=True)

    # Step 5: Get total unique items per org
    total_items = avg_error_per_item.groupby("org")["item_code"].nunique().reset_index()
    total_items.rename(columns={"item_code": "total_item"}, inplace=True)

    # Step 6: Merge total item count
    summary_df = summary_df.merge(total_items, on="org", how="left")

    # Step 7: Calculate percentage of items in each bucket
    summary_df["percent_item"] = round((summary_df["no_unique_item"] / summary_df["total_item"]) * 100, 1)

    # Step 8: Compute cumulative sum of items
    summary_df["cumulative_item"] = summary_df.groupby("org")["no_unique_item"].cumsum()
    summary_df["cumulative_percent"] = round((summary_df['cumulative_item']/summary_df['total_item'])*100, 1)

    return summary_df

In [60]:
final_summary = pd.concat([summarize_items_by_error(df) for df in results_df_list], ignore_index=True)

In [61]:
final_summary.to_csv('4_week_lead_time/final_summary_4.csv', index=False)

In [62]:
final_summary[final_summary["org"]=="P01"]

,org,pe_bucket,no_unique_item,total_item,percent_item,cumulative_item,cumulative_percent
0,P01,0-5%,72,77,93.5,72,93.5
1,P01,5-10%,1,77,1.3,73,94.8
2,P01,10-15%,0,77,0.0,73,94.8
3,P01,15-20%,1,77,1.3,74,96.1
4,P01,20-25%,0,77,0.0,74,96.1
5,P01,25-30%,0,77,0.0,74,96.1
6,P01,30-35%,0,77,0.0,74,96.1
7,P01,35-40%,0,77,0.0,74,96.1
8,P01,40-45%,0,77,0.0,74,96.1
9,P01,45-50%,0,77,0.0,74,96.1


### Roll up error metrics at item level

In [63]:
final_summary_rolled = result_df_combined.copy()
final_summary_rolled['org'] = "DUMMY"
final_summary_rolledup = summarize_items_by_error(final_summary_rolled)
final_summary_rolledup.drop(columns=['org'], inplace=True)

In [64]:
final_summary_rolledup.to_csv(r'4_week_lead_time/final_summary_rolledup_4.csv', index=False)

In [65]:
final_summary_rolledup

,pe_bucket,no_unique_item,total_item,percent_item,cumulative_item,cumulative_percent
0,0-5%,3470,5396,64.3,3470,64.3
1,5-10%,337,5396,6.2,3807,70.6
2,10-15%,194,5396,3.6,4001,74.1
3,15-20%,74,5396,1.4,4075,75.5
4,20-25%,60,5396,1.1,4135,76.6
5,25-30%,55,5396,1.0,4190,77.7
6,30-35%,22,5396,0.4,4212,78.1
7,35-40%,24,5396,0.4,4236,78.5
8,40-45%,11,5396,0.2,4247,78.7
9,45-50%,12,5396,0.2,4259,78.9


In [66]:
# Create a PrettyTable instance
table = PrettyTable()
table.field_names = final_summary_rolledup.columns.tolist()  # Set column names

# Add rows from the DataFrame
for _, row in final_summary_rolledup.iterrows():
    table.add_row(row.tolist())

# Print the table
print(table)


+---------------+----------------+------------+--------------+-----------------+--------------------+
|   pe_bucket   | no_unique_item | total_item | percent_item | cumulative_item | cumulative_percent |
+---------------+----------------+------------+--------------+-----------------+--------------------+
|      0-5%     |      3470      |    5396    |     64.3     |       3470      |        64.3        |
|     5-10%     |      337       |    5396    |     6.2      |       3807      |        70.6        |
|     10-15%    |      194       |    5396    |     3.6      |       4001      |        74.1        |
|     15-20%    |       74       |    5396    |     1.4      |       4075      |        75.5        |
|     20-25%    |       60       |    5396    |     1.1      |       4135      |        76.6        |
|     25-30%    |       55       |    5396    |     1.0      |       4190      |        77.7        |
|     30-35%    |       22       |    5396    |     0.4      |       4212      |  